In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
# import tensorflow as tf

# tf.compat.v1.enable_eager_execution()

In [4]:
import malaya_speech
import tensorflow as tf
from malaya_speech.train.model import fastsplit, fastspeech, sepformer
import malaya_speech.augmentation.waveform as augmentation
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
from glob import glob

wavs = glob('../speech/example-speaker/*.wav')
len(wavs)

8

In [6]:
import random

sr = 8000
speakers_size = 4

# noise = malaya_speech.load('noise.wav', sr = sr)[0]

def read_wav(f):
    return malaya_speech.load(f, sr = sr)


def random_sampling(s, length):
    return augmentation.random_sampling(s, sr = sr, length = length)

def to_mel(y):
    mel = malaya_speech.featurization.universal_mel(y)
    mel[mel <= np.log(1e-2)] = np.log(1e-2)
    return mel

def combine_speakers(files, n = 5, limit = 4):
    w_samples = random.sample(files, n)
    w_samples = [read_wav(f)[0] for f in w_samples]
    w_lens = [len(w) / sr for w in w_samples]
    w_lens = int(min(min(w_lens) * 1000, random.randint(2000, 10000)))
    w_samples = [random_sampling(w, length = w_lens) for w in w_samples]
    y = [w_samples[0]]
    left = w_samples[0].copy()

    combined = None

    for i in range(1, n):
        right = w_samples[i].copy()
        overlap = random.uniform(0.98, 1.0)
        print(i, overlap)
        len_overlap = int(overlap * len(right))
        minus = len(left) - len_overlap
        if minus < 0:
            minus = 0
        padded_right = np.pad(right, (minus, 0))
        left = np.pad(left, (0, len(padded_right) - len(left)))

        left = left + padded_right

        if i >= (limit - 1):
            if combined is None:
                combined = padded_right
            else:
                combined = np.pad(
                    combined, (0, len(padded_right) - len(combined))
                )
                combined += padded_right

        else:
            y.append(padded_right)

    if combined is not None:
        y.append(combined)

    for i in range(len(y)):
        if len(y[i]) != len(left):
            y[i] = np.pad(y[i], (0, len(left) - len(y[i])))
            y[i] = y[i] / np.max(np.abs(y[i]))

    left = left / np.max(np.abs(left))
    return left, y

# y, _ = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav')
# y = np.expand_dims(y, 0).astype(np.float32)
# y.shape

In [7]:
left, y = combine_speakers(wavs, 6)
len(left) / sr, len(y)

1 0.9988392928453518
2 0.9970046320818949
3 0.9927028371578088
4 0.9823108723073807
5 0.9946250195162768


(3.63975, 4)

In [8]:
config = malaya_speech.config.fastspeech_config
dim = 256
config['encoder_hidden_size'] = dim
config['decoder_hidden_size'] = dim
config['encoder_num_hidden_layers'] = 4
config['encoder_num_attention_heads'] = 4
config['max_position_embeddings'] = sr * 3
config = fastspeech.Config(vocab_size = 1, **config)

In [9]:
transformer = lambda: sepformer.Encoder_FastSpeech(config.encoder_self_attention_params)

In [10]:
model = sepformer.Model(transformer, transformer)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
X = tf.placeholder(tf.float32, [None, None, 1])
len_X = tf.placeholder(tf.int32, [None])
logits = model(X)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
estimate_source = tf.transpose(logits[:,:,:,0], [1, 0, 2])

In [13]:
C = 4
Y = tf.placeholder(tf.float32, [None, C, None])

In [15]:
loss, max_snr, _ = sepformer.calculate_loss(Y, estimate_source, len_X, C = C)

In [16]:
# X = tf.convert_to_tensor(np.expand_dims([left], axis = -1).astype(np.float32))
# logits = model(X)

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
l = sess.run(logits, feed_dict = {X: np.expand_dims([left, left], axis = -1)})
l.shape

(4, 2, 29118, 1)

In [21]:
sess.run(loss, feed_dict = {X: np.expand_dims([left], axis = -1), len_X: [len(left)], Y: [y]})

27.85617

In [23]:
saver = tf.train.Saver()

In [24]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [25]:
!ls -lh test

total 305936
-rw-r--r--  1 huseinzolkepli  staff    77B Mar 22 22:11 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   146M Mar 22 22:11 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    12K Mar 22 22:11 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   3.0M Mar 22 22:11 model.ckpt.meta


In [26]:
# tf.trainable_variables()